In [134]:
import uuid
from datetime import datetime

from bintrees import FastRBTree

actions = {"create":0,"del":1,"append":2}
sides = {"bid":0, "ask":1}
# Tick inputs: action, price, qty, timestamp, side

In [137]:
t1 = Tick("create",10,100,datetime.now(),"bid")
t2 =Tick("create",11,200,datetime.now(),"bid")
t3 =Tick("create",11,500,datetime.now(),"ask")

In [142]:
lob1 = LOB()
lob1.process(t1)
lob1.process(t2)
lob1.process(t3)

Inserting Order
printing  bid  tree: 
Tree: 
10
volume:  100

Inserting Order
printing  bid  tree: 
Tree: 
10
11
volume:  300

Inserting Order
printing  ask  tree: 
Tree: 
11
volume:  500



In [130]:
class Tick(object):
    def __init__(self, action, price, qty, timestamp, side):
        self.action = action
        self.price = price
        self.qty = qty
        self.timestamp = timestamp
        self.side = sides[side]
        self.id = uuid.uuid1()

In [141]:
class LOB(object):
    def __init__(self):
        self.bids = Tree()
        self.asks = Tree()
        self.last_tick = None
        self.last_timestamp = 0

    def process(self, tick):
        if tick.side == 0:
            tree = self.bids
        else:
            tree = self.asks
            
        action = actions[tick.action]
        
        if action == 0:
            print("Inserting Order")
            tree.insert(tick)
        elif action == 1:
            print("Removing Order")
            tree.remove(tick.id)
        elif action == 2:
            print("Updating Order")
            tree.update(tick)
        else:
            print("Unknown Action")
        print("printing ",list(sides.keys())[list(sides.values()).index(tick.side)]," tree: ")
        print(tree.print_tree())



In [100]:

class Tree(object):
    def __init__(self):
        self.price_tree = FastRBTree()
        self.volume = 0
        self.price_map = {}
        self.order_map = {}
        self.min_price = None
        self.max_price = None
        
    def print_tree(self):
        print("Tree: ")
        [print(i) for i in self.price_tree]
        print("volume: ", self.volume)
#         print("prices: ")
#         [print(i) for i in self.price_map]
#         print("orders: ")
#         [print(i) for i in self.order_map]
        return ""


    def get_price(self, price):
        return self.price_map[price]

    def get_order(self, id):
        return self.order_map[id]

    def create_price(self, price):
        new_list = OrderList()
        self.price_tree.insert(price, new_list)
        self.price_map[price] = new_list
        if self.max_price == None or price > self.max_price:
            self.max_price = price
        if self.min_price == None or price < self.min_price:
            self.min_price = price

    def remove_price(self, price):
        self.price_tree.remove(price)
        del self.price_map[price]

        if self.max_price == price:
            try:
                self.max_price = max(self.price_tree)
            except ValueError:
                self.max_price = None
        if self.min_price == price:
            try:
                self.min_price = min(self.price_tree)
            except ValueError:
                self.min_price = None

    def price_exists(self, price):
        return price in self.price_map

    def order_exists(self, id):
        return id in self.order_map

    def insert(self, new_tick):
        if new_tick.price not in self.price_map:
            self.create_price(new_tick.price)
        order = Order(new_tick, self.price_map[new_tick.price])
        self.price_map[order.tick.price].append_order(order)
        self.order_map[order.tick.id] = order
        self.volume += order.tick.qty

    def update(self, new_tick):
        original_order = self.order_map[new_tick.id]
        original_volume = original_order.tick.qty
        if new_tick.price != original_order.tick.price:
            order_list = self.price_map[original_order.tick.price]
            order_list.remove_order(original_order)
            if len(order_list) == 0:
                self.remove_price(original_order.tick.price)
                self.volume -= original_volume
            self.insert_tick(tick)
        else:
            original_order.update_qty(tick.qty, tick.price)
            self.volume += order.price.qty - original_volume

    def remove(self, id):
        order = self.order_map[id]
        self.volume -= order.tick.qty
        order.order_list.remove_order(order)
        if len(order.order_list) == 0:
            self.remove_price(order.tick.price)
        del self.order_map[id]

    def get_max(self):
        return self.max_price

    def get_min(self):
        return self.min_price




In [32]:
class OrderList(object):
    def __init__(self):
        self.head_order = None
        self.tail_order = None
        self.length = 0
        self.volume = 0
        self.last = None
        
    def __len__(self):
        return self.length
    
    def append_order(self, order):
        if len(self) == 0:
            order.next_order = None
            order.prev_order = None
            self.head_order = order
            self.tail_order = order
        else:
            order.prev_order = self.tail_order
            order.next_order = None
            self.tail_order.next_order = order
            self.tail_order = order
        self.length += 1
        self.volume += order.tick.qty

    def remove_order(self, order):
        self.volume -= order.qty
        self.length -= 1
        if len(self) == 0:
            return
        next_order = order.next_order
        prev_order = order.prev_order
        if next_order != None and prev_order != None:
            next_order.prev_order = prev_order
            prev_order.next_order = next_order
        elif next_order != None:
            next_order.prev_order = None
            self.head_order = next_order
        elif prev_order != None:
            prev_order.next_order = None
            self.tail_order = prev_order

    def move_tail(self, order):
        if order.prev_order != None:
            order.prev_order.next_order = self.next_order
        else:
            self.head_order = order.next_order
        order.next_order.prev_order = order.prev_order
        self.tail_order.next_order = order
        self.tail_order = order
        order.prev_order = self.tail_order
        order.next_order = None



In [111]:

class Order(object):
    def __init__(self, tick, order_list):
        self.tick = tick
        self.order_list = order_list
        self.next_order = None
        self.prev_order = None

    def get_next_order(self):
        return self.next_order

    def get_prev_order(self):
        return self.prev_order

    def update_qty(self, new_qty, new_timestamp):
        if new_qty > self.qty and self.order_list.tail_order != self:
            self.order_list.move_tail(self)
        self.order_list.volume -= self.qty - new_qty
        self.tick.timestamp = new_timestamp
        self.tick.qty = new_qty


    def get_price(self):
        return self.tick.price

    def get_qty(self):
        return self.tick.qty

    def get_id(self):
        return self.tick.id

    def get_side(self):
        return self.tick.side